<a href="https://colab.research.google.com/github/PRAGHATHAPRIYA2589/step/blob/Day-5/Day_5_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [3]:
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/keww1ps.txt',
    block_size=64
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
pds_data[0], pds_data[0].shape

(tensor([   54,   440, 28290, 21881,  5258,   198, 32541, 39453,  3268, 12964,
          8763, 18422,  7473, 42715, 30554,   198,     7,    36, 16779,  9306,
         37384,  5188,     8,   198,   198,  1238,  1731,    12,  1495,   628,
           200, 11442,  4524,   784,   370,  8874,  1677, 21881,  5258,   198,
           198,  1797, 15766,  9773,    12,  4524,  1120,    12, 47396,    12,
            23,   198,   198,  8206,  2070,   329,  5016, 30554,   198,  5962,
          5061,   198, 16192,  4793]),
 torch.Size([64]))

In [5]:
print(tokenizer.decode(pds_data[0]))

W OVEN WORDS
TEXTBOOK IN ENGLISH FOR CLASS XI
(ELECTIVE COURSE)

2024-25

11074 – WOVEN WORDS

ISBN 81-7450-514-8

Textbook for Class XI
First Edition
March 2006


In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [7]:
collator_example = data_collator([tokenizer('I am an input'), tokenizer('So am I')])

collator_example

{'input_ids': tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]]), 'labels': tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])}

In [8]:
collator_example.input_ids

tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]])

In [9]:
tokenizer.pad_token_id

50256

In [10]:
collator_example.attention_mask

tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])

In [11]:
collator_example.labels

tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])

In [12]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

pretrained_generator = pipeline(
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [13]:
print('----------')
for generated_sequence in pretrained_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
This dataset shows the relationship between diet in the United States and obesity. The results of a national-level study show that weight loss of only one to three days per week is associated with a significantly higher risk of obesity. But, weight loss can increase
----------
This dataset shows the relationship between the three components of the population growth rates observed in some countries in the past.

The data has been presented as a guide, but we think the relationship between growth rates and the quality of education is very strong.
----------
This dataset shows the relationship across time within each period. In line with previous work, we have created the first large dataset of the form below that compares the temporal trend over different periods over the entire history of the state.

Note that the time
----------


In [14]:
training_args = TrainingArguments(
    output_dir="./gpt2_pds",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'eval_loss': 4.638900279998779,
 'eval_model_preparation_time': 0.0031,
 'eval_runtime': 5.5468,
 'eval_samples_per_second': 2.344,
 'eval_steps_per_second': 0.18}

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,4.158151,0.003100
2,No log,3.995597,0.003100
3,No log,3.957996,0.003100


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=6, training_loss=4.578890164693196, metrics={'train_runtime': 486.2903, 'train_samples_per_second': 0.315, 'train_steps_per_second': 0.012, 'total_flos': 4997210112000.0, 'train_loss': 4.578890164693196, 'epoch': 3.0})

In [16]:
trainer.evaluate()

{'eval_loss': 3.957995891571045,
 'eval_model_preparation_time': 0.0031,
 'eval_runtime': 4.5749,
 'eval_samples_per_second': 2.842,
 'eval_steps_per_second': 0.219,
 'epoch': 3.0}

In [17]:
trainer.save_model()

In [18]:
loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_pds')

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

Device set to use cpu


In [19]:
print('----------')
for generated_sequence in finetuned_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

----------
This dataset shows the relationship between race and ethnicity for the entire UK during the period 1978-1982. The distribution of ethnicity is consistent in the UK with the mean difference between the black, Asian, and European mean. The mean differences between the different groups
----------
This dataset shows the relationship between the average and the average value for each of the three indicators, including food cost, alcohol retail price, and social status at time of assessment. In contrast to the previous study, which examined the data of only seven national
----------
This dataset shows the relationship between the total number of children who receive schooling in a classroom and subsequent earnings of all children between the ages of 1 and 2. This data is based on the children's performance in the 2011–2013 school year in Table 1
----------
